In [1]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
import chromedriver_autoinstaller
import requests
import time
import re
from datetime import datetime

In [2]:
class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
        #options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(10)
        driver.set_window_size(1600, 1100)

        return driver

In [3]:
driver = Driver.driver_init()
article_list = []
date_list = []
driver.get('https://www.wto.org/english/news_e/news23_e/news23_e.htm')
while True:
    max_height = driver.execute_script('return document.body.scrollHeight')
    max_try = max_height//900
    y_location = 0
    for m in range(max_try):
        y_location+=900
        time.sleep(1)
        driver.execute_script(f"window.scrollTo(0, {y_location})")
    articles = driver.find_elements('xpath', '//div[@class="centerCol"]/div/div[2]/ul/li/a[contains(., "Press release") or contains(., "News items")]')
    article_urls = [i.get_attribute('href') for i in articles]
    article_list.extend(article_urls)
    break

In [ ]:
for idx, article in enumerate(article_list):
    driver.get(article)
    date_element = driver.find_element('xpath', '//div[@class="introTextDiv"]/p[1]').get_attribute('innerText')
    date_element = date_element.replace('\xa0', '-')
    date_format = "%d-%B-%Y"
    parsed_date = datetime.strptime(date_element, date_format)
    date = parsed_date.strftime("%B, %d, %Y")
    title = driver.find_element('xpath', '//div[@class="introTextDiv"]/h1').get_attribute('innerText')
    main_1 = driver.find_element('xpath', '//div[@class="introTextDiv"]/p[@class="paralargetext"][1]').get_attribute('innerText')
    main_2 = driver.find_element('xpath', '//div[@class="mainDiv"]/div[@class="centerCol"]').get_attribute('innerText')

    file_name = f'WTO_news_{idx}.txt'

    with open(file_name, 'w', encoding = 'utf-8') as file:
        file.write('<Date>' + '\n')
        file.write(date + '\n')
        file.write('<Title>' + '\n')
        file.write(title + '\n')
        file.write('<Content>' + '\n')
        #file.write(header_desc + '\n')
        file.write(main_1 + '\n')
        file.write(main_2)